In [1]:
import pandas as pd
import sqlite3

# Задание 1

In [2]:
#таблицы для задания по SQL
customers = pd.read_excel('Customer_info.xlsx')
transactions = pd.read_excel('Transactions_info.xlsx')

In [3]:
transactions.sample()

,date_new,Id_check,ID_client,Count_products,Sum_payment
108868,2016-06-01,2530069,193078,1.0,3.29


In [4]:
customers.sample()

,Id_client,Total_amount,Gender,Age,Count_city,Response_communcation,Communication_3month,Tenure
988,212850,10090.81,F,60.0,1,0,1,11


In [5]:
#создадим подключение и БД
con = sqlite3.connect(':memory:')
query = """
 CREATE TABLE test
 (a VARCHAR(20), b VARCHAR(20),
 c REAL, d INTEGER
 );"""
con.execute(query)
con.commit()

In [6]:
#проверим, что все работает и подключается
query = """
 select * from test limit 1"""
cursor = con.execute(query)
rows = cursor.fetchall()
rows

[]

In [7]:
#зальем наши таблицы в БД
create_customers = """
  CREATE TABLE customers
  (
    Id_client INTEGER,
    Total_amount REAL,
    Gender	VARCHAR(2),
    Age	REAL,
    Count_city	INTEGER,
    Response_communcation	INTEGER,
    Communication_3month INTEGER,
    Tenure INTEGER
  );"""
create_transactions = """
  CREATE TABLE transactions
  (
    date_new REAL,
    Id_check	INTEGER,
    ID_client	VARCHAR(12),
    Count_products REAL,
    Sum_payment REAL
  );"""

In [8]:
for q in [create_customers, create_transactions]:
    con.execute(q)
    con.commit()

In [9]:
customers.to_sql('customers',con,if_exists='replace',index=False)
con.commit()

In [10]:
transactions.to_sql('transactions', con, if_exists='replace', index=False)
con.commit()

In [11]:
query = """
 select *
 from customers
 limit 1"""
res = con.execute(query)
res.fetchall()

[(16052, 1636.81, 'F', 65.0, 1, 1, 1, 5)]

In [12]:
query = """
 select *
 from transactions
 limit 1"""
res = con.execute(query)
res.fetchall()

[('2016-03-01 00:00:00', 2271145, 104027, 2.034, 23.31)]

## Задача 1

Вывести список клиентов с непрерывной историей за год, средний чек за
период, средняя сумма покупок за месяц, количество всех операций по
клиенту за период

Под непрерывностью будем подразумевать минимум 1 операцию в каждом месяце в течении периода


In [13]:
query = """
with bills AS
  (
    select
      ID_client, Id_check, date_new,
      sum(Sum_payment) as revenue
    from
      transactions
    group by ID_client, Id_check, date_new
  )
select
  b.ID_client,
  round(avg(b.revenue), 2) aov,
  round(avg(mnt.total_mnt), 2) avg_month,
  count(distinct(b.Id_check)) cnt_bills,
  count(distinct(b.date_new)) cnt_mnt
from bills b
join
  (
    select ID_client, date_new, sum(revenue) total_mnt from bills
    group by ID_client, date_new
  ) as mnt on mnt.ID_client = b.ID_client
where b.ID_client in
  (select
    ID_client
  from
    transactions
  group by ID_client
  having count(distinct(date_new)) = 13)
group by b.ID_client
 limit 10"""
res = con.execute(query)
names = list(map(lambda x: x[0], res.description))
pd.DataFrame(res.fetchall(), columns = names)

,ID_client,aov,avg_month,cnt_bills,cnt_mnt
0,16052,81.28,44898.32,7181,13
1,185122,69.76,177.07,33,13
2,185151,91.31,273.92,39,13
3,185156,178.49,823.78,60,13
4,185348,96.97,648.95,87,13
5,185364,179.26,1792.60,130,13
6,185389,87.63,559.47,83,13
7,185435,73.27,180.37,32,13
8,185459,97.76,857.31,114,13
9,185503,101.88,681.80,87,13


## Задача 2

Вывести помесячную информацию: средняя сумма чека в месяц, среднее
количество операций в месяц, среднее количество клиентов, которые
совершали операции; долю от общего количества операций за год и долю в
месяц от общей суммы операций; вывести % соотношение M/F/NA в каждом
месяце с их долей затрат

In [14]:
query = """
with month_bills AS
  (
    select
      t.ID_client, c.Gender, t.Id_check, t.date_new,
      sum(t.Sum_payment) as revenue,
      1 num_bill
    from
      transactions t
    join customers c on c.ID_client = t.ID_client
    group by t.ID_client, c.Gender, t.Id_check, t.date_new
  )
select
  month, aov, cnt_bills, cnt_customers,
  round(cast(cnt_bills as real) / sum(cnt_bills) over (), 3) bills_share_of_year,
  round(month_revenue / sum(month_revenue) over (), 3) revenue_share_of_year,
  round(male_revenue / month_revenue, 3) male_revenue_share,
  round(female_revenue / month_revenue, 3) female_revenue_share,
  round(NA_revenue / month_revenue, 3) NA_revenue_share
from
  (select
    date(date_new) month,
    sum(revenue) month_revenue,
    round(avg(revenue), 2) aov,
    sum(num_bill) cnt_bills,
    count(distinct(ID_client)) cnt_customers,
    round(sum(case when Gender = 'M' then revenue else 0 end), 2) male_revenue,
    round(sum(case when Gender = 'F' then revenue else 0 end), 2) female_revenue,
    round(sum(case when Gender is null then revenue else 0 end), 2) NA_revenue
  from month_bills
  group by date_new
  )t

"""
res = con.execute(query)
names = list(map(lambda x: x[0], res.description))
pd.DataFrame(res.fetchall(), columns = names)


,month,aov,cnt_bills,cnt_customers,bills_share_of_year,revenue_share_of_year,male_revenue_share,female_revenue_share,NA_revenue_share
0,2015-06-01,95.26,316,224,0.008,0.008,0.253,0.725,0.022
1,2015-07-01,93.85,2929,939,0.070,0.069,0.246,0.733,0.021
2,2015-08-01,91.48,2862,907,0.068,0.066,0.214,0.762,0.024
3,2015-09-01,93.24,2794,901,0.067,0.065,0.224,0.749,0.027
4,2015-10-01,94.20,2936,967,0.070,0.070,0.246,0.733,0.021
5,2015-11-01,90.30,2794,918,0.067,0.063,0.242,0.734,0.024
6,2015-12-01,91.64,3139,1032,0.075,0.072,0.271,0.705,0.025
7,2016-01-01,90.18,3052,991,0.073,0.069,0.266,0.708,0.026
8,2016-02-01,103.07,4681,1254,0.112,0.121,0.264,0.708,0.028
9,2016-03-01,95.85,4467,1181,0.106,0.108,0.248,0.722,0.030


## Задача 3
Вывести возрастные группы клиентов с шагом 10 лет и отдельно клиентов,
у которых нет данной информации с параметрами сумма и количество
операций за весь период, и поквартально, средние показатели и %.

In [15]:
query = """
select
  age_group, size_group, revenue, cnt_bills,
  revenue / cnt_bills aov,
  revenue / sum(revenue) over () share_revenue,
  cnt_bills / sum(cast(cnt_bills as real)) over() share_bills,
  revenue_2015_2, cnt_bills_2015_2,
  revenue_2015_3, cnt_bills_2015_3,
  revenue_2015_4, cnt_bills_2015_4,
  revenue_2016_1, cnt_bills_2016_1,
  revenue_2016_2, cnt_bills_2016_2,
  (revenue_2015_2 + revenue_2015_3 + revenue_2015_4 + revenue_2016_1 + revenue_2016_2) / 5 avg_revenue_q,
  (cnt_bills_2015_2 + cnt_bills_2015_3 + cnt_bills_2015_4 + cnt_bills_2016_1 + cnt_bills_2016_2) / 5 avg_bills_q
from
  (select
    age_group,
    count(distinct(Id_client)) size_group,
    sum(Sum_payment) revenue,
    count(distinct(Id_check)) cnt_bills,
    sum(case when year || '_' || q = '2015_2' then Sum_payment else 0 end) revenue_2015_2,
    count(distinct case when year || '_' || q = '2015_2' then Id_check else null end) cnt_bills_2015_2,
    sum(case when year || '_' || q = '2015_3' then Sum_payment else 0 end) revenue_2015_3,
    count(distinct case when year || '_' || q = '2015_3' then Id_check else null end) cnt_bills_2015_3,
    sum(case when year || '_' || q = '2015_4' then Sum_payment else 0 end) revenue_2015_4,
    count(distinct case when year || '_' || q = '2015_4' then Id_check else null end) cnt_bills_2015_4,
    sum(case when year || '_' || q = '2016_1' then Sum_payment else 0 end) revenue_2016_1,
    count(distinct case when year || '_' || q = '2016_1' then Id_check else null end) cnt_bills_2016_1,
    sum(case when year || '_' || q = '2016_2' then Sum_payment else 0 end) revenue_2016_2,
    count(distinct case when year || '_' || q = '2016_2' then Id_check else null end) cnt_bills_2016_2
    from (
      select
        c.Id_client, c.age, c.age_group,
        t.date_new,
        (strftime('%m', t.date_new) + 2) / 3 q,
        strftime('%Y', t.date_new)  year,
        t.Id_check, t.Sum_payment
      from transactions t
      join
        (select  Id_client, age,
            coalesce(
              substr(cast(cast(Age as real) / 10
                  as varchar), 1, 1), 'blank') age_group
          from customers
          group by Id_client
          order by Id_client)
      c on t.ID_client = c.Id_client
      ) tbl
  group by age_group
  )res
"""
res = con.execute(query)
names = list(map(lambda x: x[0], res.description))
pd.DataFrame(res.fetchall(), columns = names)

,age_group,size_group,revenue,cnt_bills,aov,share_revenue,share_bills,revenue_2015_2,cnt_bills_2015_2,revenue_2015_3,cnt_bills_2015_3,revenue_2015_4,cnt_bills_2015_4,revenue_2016_1,cnt_bills_2016_1,revenue_2016_2,cnt_bills_2016_2,avg_revenue_q,avg_bills_q
0,0,11,12047.81,118,102.100085,0.003029,0.002812,88.11,1,1973.71,20,3436.17,38,4471.20,36,2078.62,23,2409.562,23
1,1,84,131021.16,1330,98.512150,0.032939,0.031692,797.19,7,22948.07,262,28742.70,304,40591.09,389,37942.11,368,26204.232,266
2,2,570,823043.48,8175,100.678102,0.206913,0.194801,5921.80,61,150574.16,1580,171459.47,1760,241356.11,2323,253731.94,2451,164608.696,1635
3,3,560,781668.13,8171,95.663705,0.196511,0.194705,5608.06,57,154689.15,1625,163868.04,1770,235318.03,2413,222184.85,2306,156333.626,1634
4,4,437,526764.92,5480,96.124985,0.132428,0.130582,4678.59,44,118069.13,1213,103329.21,1135,150798.95,1535,149889.04,1553,105352.984,1096
5,5,410,560976.97,5694,98.520718,0.141029,0.135681,3105.49,36,103928.50,1081,117123.27,1231,171033.90,1629,165785.81,1717,112195.394,1138
6,6,231,919942.23,10794,85.227185,0.231273,0.257208,7496.51,90,200081.32,2349,183666.22,2149,281201.18,3279,247497.00,2927,183988.446,2158
7,7,85,150486.83,1418,106.126114,0.037832,0.033789,1894.81,15,32409.33,313,31423.58,319,38157.81,367,46601.30,404,30097.366,283
8,8,6,7140.84,79,90.390380,0.001795,0.001882,100.82,1,1073.05,13,2025.16,23,2650.15,25,1291.66,17,1428.168,15
9,blank,35,64641.24,707,91.430325,0.016251,0.016847,409.78,4,11477.04,129,11448.64,140,20265.34,204,21040.44,230,12928.248,141
